In [3]:
with open('C:/Users/dnjse/OneDrive/바탕 화면/국토교통/기상코드/nxny좌표.txt','r') as f:
    lst = [line.strip().replace(',','').split(' ') for line in f]

In [17]:
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
import urllib
import requests
import json
import pandas as pd
import datetime

# 기상청_동네 예보 조회 서비스 api 데이터 url 주소, 초단기이기때문에 getUltraSrtFcst 사용
url = "https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtFcst"

serviceKey = "RvvK7UoRrYoy9KAatmtUewyyjXr0eNKHg45pgZPzLM6T1pzDjD4HEIdPVdI822q26KJWvc59QQDa%2BpjZvFEXaA%3D%3D"  # 공공데이터 포털에서 생성된 본인의 서비스 키를 복사 / 붙여넣기
serviceKeyDecoded = unquote(serviceKey, 'UTF-8')  # 공데이터 포털에서 제공하는 서비스키는 이미 인코딩된 상태이므로, 디코딩하여 사용해야 함

now = datetime.datetime.now()
today = datetime.datetime.today().strftime("%Y%m%d")
y = datetime.date.today() - datetime.timedelta(days=1)
yesterday = y.strftime("%Y%m%d")

weather_data_list = []

for i in lst[:10]:
    ny, nx = int(i[0]), int(i[1])
    print(ny, nx)
    if now.minute < 45:  # base_time와 base_date 구하는 함수
        if now.hour == 0:
            base_time = "2330"
            base_date = yesterday
        else:
            pre_hour = now.hour - 1
            if pre_hour < 10:
                base_time = "0" + str(pre_hour) + "30"
            else:
                base_time = str(pre_hour) + "30"
            base_date = today
    else:
        if now.hour < 10:
            base_time = "0" + str(now.hour) + "30"
        else:
            base_time = str(now.hour) + "30"
    base_date = today

    queryParams = '?' + urlencode({quote_plus('serviceKey'): serviceKeyDecoded, quote_plus('base_date'): base_date,
                                   quote_plus('base_time'): base_time, quote_plus('nx'): nx, quote_plus('ny'): ny,
                                   quote_plus('dataType'): 'json',
                                   quote_plus('numOfRows'): '60'})  # 페이지로 안나누고 한번에 받아오기 위해 numOfRows=60으로 설정해주었다

    # 값 요청 (웹 브라우저 서버에서 요청 - url주소와 파라미터)
    res = requests.get(url + queryParams)  # verify=False이거 안 넣으면 에러남ㅜㅜ
    items = res.json().get('response').get('body').get('items')  # 데이터들 아이템에 저장
    # print(items)# 테스트

    weather_data = dict()
    for item in items['item']:
        # 기온
        if item['category'] == 'T1H':
            weather_data['tmp'] = item['fcstValue']
        # 습도
        if item['category'] == 'REH':
            weather_data['hum'] = item['fcstValue']
        # 하늘상태: 맑음(1) 구름많은(3) 흐림(4)
        if item['category'] == 'SKY':
            weather_data['sky'] = item['fcstValue']
        # 1시간 동안 강수량
        if item['category'] == 'RN1':
            weather_data['rain'] = item['fcstValue']
        # 강수형태
        if item['category'] == 'PTY':
            weather_data['강수형태'] = item['fcstValue']
        #풍향
        if item['category'] == 'VEC':
            weather_data['풍향'] = item['fcstValue']+(' 도')
        #풍속
        if item['category'] == 'WSD':
            weather_data['풍속'] = item['fcstValue']+(' m/s')

    if int(weather_data['sky']) <= 5:
        weather_data['sky'] = ("맑음")
    elif int(weather_data['sky']) <= 8:
        weather_data['sky'] = ("구름많음")
    else:
        weather_data['sky'] = ("흐림")


    weather_data_list.append(weather_data)
    
max_retries = 100

for retry in range(max_retries):
    try:
        # 값 요청 (웹 브라우저 서버에서 요청 - url주소와 파라미터)
        res = requests.get(url + queryParams, timeout=10)  # 타임아웃을 10초로 설정
        res.raise_for_status()  # 응답 코드를 확인하여 오류가 있는지 확인
        items = res.json().get('response').get('body').get('items')  # 데이터들 아이템에 저장

        # 이후 코드는 그대로 유지
        # ...

        break  # 성공적으로 데이터를 받아왔으면 반복문 종료

    except requests.exceptions.RequestException as e:
        print(f"Error occurred during data request. Retrying ({retry + 1}/{max_retries})...")
        continue

53 37
56 39
52 38
48 35
50 36
60 38
52 38
51 37
57 39
52 38


In [18]:
df = pd.DataFrame(weather_data_list)
df

,강수형태,rain,sky,tmp,hum,풍향,풍속
0,1,2.0mm,맑음,25,95,196 도,7 m/s
1,1,2.0mm,맑음,24,95,196 도,7 m/s
2,1,2.0mm,맑음,25,95,196 도,7 m/s
3,1,2.0mm,맑음,26,95,195 도,8 m/s
4,1,2.0mm,맑음,25,95,196 도,8 m/s
5,1,3.0mm,맑음,24,95,194 도,6 m/s
6,1,2.0mm,맑음,25,95,196 도,7 m/s
7,1,2.0mm,맑음,25,95,197 도,8 m/s
8,1,1.0mm,맑음,24,95,196 도,7 m/s
9,1,2.0mm,맑음,25,95,196 도,7 m/s


In [19]:
ny = [int(i[0]) for i in lst]
nx = [int(i[1]) for i in lst]
d = dict()
d['ny'] = ny
d['nx'] = nx

In [27]:
nynx = [str(i[0])+'_'+str(i[1]) for i in lst]

In [20]:
df['ny'] = d['ny'][:10]
df['nx'] = d['nx'][:10]
df

,강수형태,rain,sky,tmp,hum,풍향,풍속,ny,nx
0,1,2.0mm,맑음,25,95,196 도,7 m/s,53,37
1,1,2.0mm,맑음,24,95,196 도,7 m/s,56,39
2,1,2.0mm,맑음,25,95,196 도,7 m/s,52,38
3,1,2.0mm,맑음,26,95,195 도,8 m/s,48,35
4,1,2.0mm,맑음,25,95,196 도,8 m/s,50,36
5,1,3.0mm,맑음,24,95,194 도,6 m/s,60,38
6,1,2.0mm,맑음,25,95,196 도,7 m/s,52,38
7,1,2.0mm,맑음,25,95,197 도,8 m/s,51,37
8,1,1.0mm,맑음,24,95,196 도,7 m/s,57,39
9,1,2.0mm,맑음,25,95,196 도,7 m/s,52,38


In [28]:
df['ny_nx'] = nynx[:10]
df

,강수형태,rain,sky,tmp,hum,풍향,풍속,ny,nx,ny_nx
0,1,2.0mm,맑음,25,95,196 도,7 m/s,53,37,53_37
1,1,2.0mm,맑음,24,95,196 도,7 m/s,56,39,56_39
2,1,2.0mm,맑음,25,95,196 도,7 m/s,52,38,52_38
3,1,2.0mm,맑음,26,95,195 도,8 m/s,48,35,48_35
4,1,2.0mm,맑음,25,95,196 도,8 m/s,50,36,50_36
5,1,3.0mm,맑음,24,95,194 도,6 m/s,60,38,60_38
6,1,2.0mm,맑음,25,95,196 도,7 m/s,52,38,52_38
7,1,2.0mm,맑음,25,95,197 도,8 m/s,51,37,51_37
8,1,1.0mm,맑음,24,95,196 도,7 m/s,57,39,57_39
9,1,2.0mm,맑음,25,95,196 도,7 m/s,52,38,52_38


In [31]:
df.to_csv('./n좌표별_기상정보.csv', index=False, encoding = 'ANSI')

In [11]:
df_center = pd.read_csv('C:/Users/dnjse/OneDrive/바탕 화면/국토교통/qgis/격자중심좌표/격자중심_위경도좌표.csv', encoding = 'ANSI')
df_center

,gid,lbl,val,x,y
0,다나134987,6.77,6.77,126.568468,33.477069
1,다다279074,5.19,5.19,126.723816,33.556607
2,다다055009,8.94,8.94,126.483220,33.496241
3,나나873852,6.90,6.90,126.289295,33.352900
4,나나974934,6.90,6.90,126.396892,33.427860
...,...,...,...,...,...
99767,나나948884,NaN,NaN,126.369513,33.382518
99768,나나782863,NaN,NaN,126.191365,33.361827
99769,다다307064,NaN,NaN,126.754056,33.547774
99770,다다167020,NaN,NaN,126.603675,33.507092


In [29]:
df_center['ny'] = d['ny']
df_center['nx'] = d['nx']
df_center['ny_nx'] = nynx
df_center

,gid,lbl,val,x,y,ny,nx,ny_nx
0,다나134987,6.77,6.77,126.568468,33.477069,53,37,53_37
1,다다279074,5.19,5.19,126.723816,33.556607,56,39,56_39
2,다다055009,8.94,8.94,126.483220,33.496241,52,38,52_38
3,나나873852,6.90,6.90,126.289295,33.352900,48,35,48_35
4,나나974934,6.90,6.90,126.396892,33.427860,50,36,50_36
...,...,...,...,...,...,...,...,...
99767,나나948884,NaN,NaN,126.369513,33.382518,50,35,50_35
99768,나나782863,NaN,NaN,126.191365,33.361827,47,35,47_35
99769,다다307064,NaN,NaN,126.754056,33.547774,57,39,57_39
99770,다다167020,NaN,NaN,126.603675,33.507092,54,38,54_38


In [30]:
df_center.to_csv('C:/Users/dnjse/OneDrive/바탕 화면/국토교통/qgis/격자중심좌표/격자중심_nxny좌표.csv', index = False)